In [1]:
import pandas as pd 
import os
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier 
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [2]:
#read dataset 
df=pd.read_csv("cleaned_dataset.csv")
df.head(5)

,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Credit_Amount,Loan_Annuity,Client_Education,Client_Gender,Loan_Contract_Type,...,ID_Days,Client_Family_Members,Cleint_City_Rating,Score_Source_2,Score_Source_3,Phone_Change,Default,balance_ratio,avg_source_score,capablity
0,6750.0,0.0,0.0,1.0,0.0,61190.55,3416.85,4,1,0,...,383.0,2.0,2.0,0.478787,0.478787,63.0,0,9.065267,0.478787,0.506200
1,18000.0,0.0,0.0,1.0,0.0,59527.35,2788.20,1,1,0,...,331.0,2.0,2.0,0.552795,0.329655,277.0,0,3.307075,0.441225,0.154900
2,15750.0,0.0,0.0,1.0,1.0,53870.40,2295.45,4,1,0,...,775.0,2.0,3.0,0.135182,0.631355,1700.0,0,3.420343,0.383268,0.145743
3,33750.0,1.0,0.0,1.0,0.0,133988.40,3547.35,4,0,0,...,4043.0,4.0,1.0,0.301182,0.355639,674.0,0,3.970027,0.328410,0.105107
4,11250.0,0.0,1.0,1.0,1.0,13752.00,653.85,4,0,0,...,3910.0,2.0,2.0,0.697928,0.420611,739.0,0,1.222400,0.559269,0.058120


In [3]:
## Segregation of dependent and independent features
y=df['Default']
X=df.drop('Default',axis=1)

In [4]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=30)

In [5]:
preprocessor=StandardScaler()
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [6]:
X_train.shape,X_test.shape

((57184, 24), (24508, 24))

In [7]:
counter = Counter(y_train)
# estimate scale_pos_weight value
estimate = counter[0] / counter[1]
print('Estimate: %.3f' % estimate)
#define the param grid
# Parameters of pipelines can be set using ‘__’ separated para
clf_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 0.9, 1.0]
    }
#define the pipeline
model_pipe = Pipeline([('gbtree',GradientBoostingClassifier)])
#define the classifer
clf = GridSearchCV(estimator=model_pipe, param_grid=clf_grid, cv=3,verbose=50, scoring='roc_auc', n_jobs=-1)
#fit to training data
clf.fit(X_train, y_train)

Estimate: 11.314
Fitting 3 folds for each of 729 candidates, totalling 2187 fits


In [ ]:
print("Best model parameters: \n")
print(clf.best_params_ )
print("Best model score: \n")
print(clf.best_score_)